In [1]:
import nltk
import numpy as np
from typing import List

import torch
import evaluate
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('treebank')

tagged_sentences = nltk.corpus.treebank.tagged_sents()
print('Number of samples: ', len(tagged_sentences))

sentences, sentence_tags = [], []
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append([word.lower() for word in sentence])
    sentence_tags.append([tag for tag in tags])

label2id = {}
id_counter = 0
for tags in sentence_tags:
    for tag in tags:
        if tag not in label2id:
            label2id[tag] = id_counter
            id_counter += 1
id2label = {v: k for k, v in label2id.items()}

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


Number of samples:  3914


In [3]:
train_sentences, test_sentences, train_tags, test_tags = train_test_split(
    sentences,
    sentence_tags,
    test_size=0.3
)

valid_sentences, test_sentences, valid_tags, test_tags = train_test_split(
    test_sentences,
    test_tags,
    test_size=0.5
)

In [4]:
model_name = 'QCRI/bert-base-multilingual-cased-pos-english'
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)
MAX_LEN = 256

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
class PosTaggingDataset(Dataset):
    def __init__(
        self,
        sentences: List[List[str]],
        tags: List[List[str]],
        tokenizer,
        label2id,
        max_len=MAX_LEN
    ):
        super().__init__()
        self.sentences = sentences
        self.tags = tags
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.label2id = label2id

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        input_token = self.sentences[idx]
        label_token = self.tags[idx]

        input_token = self.tokenizer.convert_tokens_to_ids(input_token)
        attention_mask = [1] * len(input_token)
        labels = [self.label2id[token] for token in label_token]

        return {
            'input_ids': self.pad_and_truncate(input_token, pad_id=self.tokenizer.pad_token_id),
            'labels': self.pad_and_truncate(labels, pad_id=0),
            'attention_mask': self.pad_and_truncate(attention_mask, pad_id=0)
        }

    def pad_and_truncate(self, inputs: List[int], pad_id: int):
        if len(inputs) < self.max_len:
            padded_inputs = inputs + [pad_id] * (self.max_len - len(inputs))
        else:
            padded_inputs = inputs[:self.max_len]
        return torch.as_tensor(padded_inputs)

In [6]:
train_dataset = PosTaggingDataset(train_sentences, train_tags, tokenizer, label2id)
val_dataset = PosTaggingDataset(valid_sentences, valid_tags, tokenizer, label2id)
test_dataset = PosTaggingDataset(test_sentences, test_tags, tokenizer, label2id)

In [7]:
accuracy = evaluate.load('accuracy')

ignore_label = len(label2id)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mask = labels != ignore_label
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions[mask], references=labels[mask])

In [8]:
training_args = TrainingArguments(
    output_dir='outd=_dir',
    report_to='none',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-8-f253e7fd8fb4>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.072894,0.981660
2,No log,0.048488,0.986964
3,0.216000,0.042382,0.988474
4,0.216000,0.039337,0.989200
5,0.216000,0.037874,0.989599
6,0.038200,0.035995,0.989885
7,0.038200,0.035459,0.990171
8,0.038200,0.035208,0.990211
9,0.030200,0.035072,0.990304
10,0.030200,0.035064,0.990311


TrainOutput(global_step=1720, training_loss=0.08626077840494555, metrics={'train_runtime': 1892.871, 'train_samples_per_second': 14.47, 'train_steps_per_second': 0.909, 'total_flos': 3579882599208960.0, 'train_loss': 0.08626077840494555, 'epoch': 10.0})

In [9]:
test_sentence = 'We are exploring the topic of deep learning'
input = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentence.split())])
input = input.to('cuda')

outputs = model(input)
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()

pred_tags = ''
for pred in preds:
    pred_tags += id2label[pred] + " "

print(pred_tags)

PRP VBP VBG DT NN IN JJ VBG 
